### Trabalho 4 - ABS
###### Grupo 19

Tiago Passos Rodrigues - A96414

### Enunciado

No contexto do sistema de travagem ABS (“Anti-Lock Breaking System”), pretende-se construir um autómato híbrido que descreva o sistema e que  possa ser usado para verificar as suas propriedades dinâmicas.

    
   - A componente discreta do autómato contém os modos:  `Start`,  `Free`,  `Stopping`, `Blocked`, e `Stopped`. No modo `Free`  não existe qualquer força de travagem; no modo `Stopping` aplica-se a força de travagem alta; no modo `Blocked` as rodas estão bloqueadas em relação ao corpo mas o veículo  move-se (i.e. derrapa); no modo `Stopped` o veículo está imobilizado.
   - A componente contínua  do autómato usa variáveis contínuas $\,V,v\,$ para descrever a  `velocidade do corpo`   e a `velocidade linear das rodas`  ambas em relação so solo.
   - Assume-se que o sistema de travagem exerce uma força de atrito proporcional à diferença das duas velocidades.  A dinâmica contínua, as equações de fluxo, está descrita  abaixo.
   - Os “switchs” são a componente de projeto deste trabalho; cabe ao aluno definir quais devem ser  de modo a que o sistema tenha um comportamento desejável: imobilize-se depressa e não “derrape” muito.
   - É imprescindível evitar que o sistema tenha “trajetórias de Zenão”. Isto é, sequências  infinitas de transições  entre dois modos em intervalos de tempo  que tendem para zero mas nunca alcançam zero.

Faça 

1. Defina um autómato híbrido que descreva a dinâmica do sistema segundo as notas abaixo indicadas e com os “switchs” por si escolhidos.
2. Modele em lógica temporal linear LT  propriedades que caracterizam o comportamento desejável do sistema. Nomeadamente 
     1. ”o veículo imobiliza-se completamente em menos de $t$ segundos” 
     2. “a velocidade $V$ diminui sempre com o tempo”.
3. Codifique em SMT’s o modelo que definiu em 1.
4. Codifique em SMT’s a verificação das propriedades temporais que definiu em 2.

### Implementação

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import *

Start = Int(-1)
Free = Int(0)
Stopping = Int(1)
Blocked = Int(2)
Stopped = Int(3)

#### Variáveis

Variáveis do autómato híbrido:
- uma variável discreta que denota o *modo de funcionamento* ($m$)
- uma variável contínua que denota a velocidade do corpo ($V$)
- uma variável contínua que denota a velocidade linear das rodas ($v$)
- uma constante que denota a força de atrito ($f$)
- variável dos segundos passados ($t$)

In [2]:
a = 0.5
a_blocked = 0.01
P = 1000
f = a * P
VelInicial = 20
limite_timer = 0.1
c_stopping = 10
c = 0.5
erro = 0.1

def declare(i):
    s = {}
    s['m'] = Symbol('m'+str(i),INT)
    s['V'] = Symbol('V'+str(i),REAL)
    s['v'] = Symbol('v'+str(i),REAL)
    s['t'] = Symbol('t'+str(i),REAL)
    s['timer'] = Symbol('timer'+str(i),REAL)
    return s

### Estado inicial

$$
m = \mathsf{Start} \wedge V = V_{inicial} \wedge v = V_{inicial} \wedge F = 0 \wedge t = 0
$$

In [3]:
def init(s):
    return And(Equals(s['m'],Start),Equals(s['V'],Real(VelInicial)),Equals(s['v'],Real(VelInicial)),Equals(s['t'],Real(0)))
s = declare(0)
print(init(s))

((m0 = -1) & (V0 = 20.0) & (v0 = 20.0) & (t0 = 0.0))


### Transições

As transições do FOTS incluem os dois tipos de transição que podem ocorrer num autómato híbrido:
- Transições *timed* descrevem os *flows* associados a cada modo (a evolução das variáveis contínuas)
- Transições *untimed* descrevem os *switches* entre modos

As transições *untimed* podem ser obtidas através de uma codificação muito directa das guardas e efeitos especificadas nos *switches*, com a restrição que o tempo não evolui nestas transições, nem as variáveis contínuas se modificam a não ser que lhes seja explicitamente atribuído um novo valor no efeito do *switch*. Por exemplo:

$$
\begin{array}{c}
m = \mathsf{Start} \wedge m' = \mathsf{Free} \wedge V' = V \wedge v' = v \wedge t' = t \wedge timer = 0 \wedge timer' = 0 \\
\vee\\
m = \mathsf{Free} \wedge m' = \mathsf{Stopping} \wedge V' = V \wedge v' = v \wedge t' = t \wedge timer \ge limitetempo \wedge timer' = 0\\
\vee\\
m = \mathsf{Stopping} \wedge m' = \mathsf{Blocked} \wedge V' = V \wedge v' = v \wedge t' = t \wedge timer' = 0\\
\vee\\
m = \mathsf{Stopping} \wedge m' = \mathsf{Stopped} \wedge V' = 0 \wedge v' = 0 \wedge t' = t \\
\vee\\ 
m = \mathsf{Blocked} \wedge m' = \mathsf{Free} \wedge V' = V \wedge v' = v \wedge t' = t \wedge t \ge limitetempo \wedge timer' = 0\\
\end{array}
$$

--------------------------------------------------------------------------------

Nas transições *timed* o modo permanece constante, mas o resto das variáveis evoluem de acordo com as restrições indicadas. Os *flows* são especificados indicando qual a derivada em relação ao tempo de cada variável contínua. Para codificar os *flows* no FOTS é necessário fazer a sua *discretização*, ou seja, indicar qual a variação ocorrida num intervalo de tempo $t'-t$. $\dot{V} = -F$ temos que $V' - V = -F(t'-t)$. 
Se a relação de *flow* é amortecida, como por exemplo no modo $\mathsf{Free}$ do carro, uma pseudo-solução seria aproximar a derivada $\dot{V}$ por $\frac{V'-V}{t'-t}$ obtendo-se
$(V' - V)/(t'-t) = -F$, mas nesta equação ocorre um produto não-escalar $\,-F*(t'-t)\,$ e, por isso,  ela não vai ser decidível.
Uma sugestão para discretizar consiste em usar um valor constante para substituir a variável $F$, inferido a partir do invariante de modo. Neste caso a equação de flow transforma-se numa inequação.


Por exemplo, como sabemos que $F = 0$ podemos substituir $F$ por 0 na equação, dando origem à relação $V' - V = -0(t'-t)$. 
No caso do *flow* associado ao modo $\mathsf{ON}$, $5\dot{x}+x=25$,  como $x \le 22$ podemos aproximar $x$ por 22, dando origem à relação $x' - x \ge \frac{3}{5}(t'-t)$. 

Finalmente, é necessário também impor os invariantes dos modos no FOTS. Isso pode ser feito acrescentando a cada transição uma restrição que obriga o invariante a ser cumprido. Temos também que assugurar que o tempo avança.

Com esta técnica, no caso do termostato teríamos as seguintes duas transições *timed*:

$$
\begin{array}{c}
m = \mathsf{Free} \wedge m' = m \wedge V' - V = -F(t'-t) \wedge v' - v = (-aP + F)(t'-t) \wedge t'>t \\
\vee\\
m = \mathsf{Stopping} \wedge m' = m \wedge x' - x \ge \frac{3}{5}(t'-t) \wedge x \le 22 \wedge x' \le 22 \wedge t' > t \\
\vee\\
m = \mathsf{Blocked} \wedge m' = m \wedge x' - x \ge \frac{3}{5}(t'-t) \wedge x \le 22 \wedge x' \le 22 \wedge t' > t
\end{array}
$$

Para reduzir os erros na verificação pode-se reduzir a granularidade da discretização subdividindo cada modo em vários sub-modos que cubram toda a gama dos valores permitidos. Por exemplo neste caso poderíamos dividir cada um dos modos em 4 sub-modos, cada um com uma variação de temperatura máxima de 1 grau, cobrindo assim toda a gama de temperaturas possíveis (entre 18 e 22 graus).

In [4]:
def trans(s,p):
    # untimed
    startfree = And(Equals(s['m'],Start), Equals(p['m'], Free), Equals(p['V'],s['V']), Equals(p['v'],s['v']), Equals(p['t'],s['t']),
                   Equals(s['timer'],Real(0)), Equals(p['timer'],Real(0)))
    
    freestopping = And(Equals(s['m'],Free), Equals(p['m'], Stopping),  Equals(p['V'],s['V']), Equals(p['v'],s['v']), Equals(p['t'],s['t']),
                      GE(s['timer'],Real(limite_timer)), Equals(p['timer'],Real(0)))
    
    stoppingblocked = And(Equals(s['m'],Stopping), Equals(p['m'], Blocked),  Equals(p['V'],s['V']), Equals(p['v'],s['v']), Equals(p['t'],s['t']),
                        Equals(p['timer'],Real(0)))
    
    stoppingstopped = And(Equals(s['m'],Stopping), Equals(p['m'], Stopped), Equals(p['V'],Real(0)), Equals(p['v'],Real(0)), Equals(p['t'],s['t']))
    
    blockedfree = And(Equals(s['m'],Blocked), Equals(p['m'], Free), Equals(p['V'],s['V']), Equals(p['v'],s['v']), Equals(p['t'],s['t']),
                      GE(s['timer'],Real(limite_timer)), Equals(p['timer'],Real(0)))
    
    #timed
    freefree = And(Equals(s['m'],Free), Equals(p['m'], Free),
                   LE(p['timer'], Real(limite_timer)),
                   Equals(p['timer'] - s['timer'] , p['t'] - s['t']),
                GT(p['t'] , s['t']),
                GE(p['V'] - s['V'], (-c*(s['V'] - s['v'])) * (p['t'] - s['t'])),
                GE(p['v'] - s['v'], (-a*P + c * (s['V'] - s['v'])) * (p['t'] - s['t'])),
                  GE(p['V'] , Real(0)),
                        GE(p['v'] , Real(0)))
                
    stoppingstopping = And(Equals(s['m'],Stopping), Equals(p['m'], Stopping),
                Equals(s['timer'], Real(0)), Equals(p['timer'], Real(0)),
                GT(p['t'],s['t']),
                GE(p['V'] - s['V'],(-c_stopping*(s['V'] - s['v'])) * (p['t'] - s['t'])),
                GE(p['v'] - s['v'] , (-a*P + c_stopping * (s['V'] - s['v'])) * (p['t'] - s['t'])),
                LE(s['v'], s['V']),
                LE(p['v'], p['V']))
    
    blockedblocked = And(Equals(s['m'], Blocked), Equals(p['m'], Blocked), LT(s['t'] , p['t']),
                         Equals(p['timer'] - s['timer'] , p['t'] - s['t']),
                        LE(p['timer'], Real(limite_timer)),
                        GE(p['v'] - s['v'], ((-a_blocked * P) * (p['t'] - s['t']))),
                        LE(s['v'], s['V']),
                        LE(p['v'], p['V']),
                        GE(p['V'] , Real(0)),
                        GE(p['v'] , Real(0)))
    
    stoppedstopped = And(Equals(s['m'], Stopped), Equals(p['m'], Stopped), Equals(s['t'], p['t']),
                        Equals(p['v'], s['v']), Equals(p['V'], s['V']), Equals(s['timer'], Real(0)),
                        Equals(p['timer'], Real(0)))
                         
    
    return Or([startfree,freestopping,stoppingblocked,stoppingstopped,blockedfree,
               freefree,stoppingstopping,blockedblocked,stoppedstopped])

### Simulação

In [5]:
def gera_traco(declare,init,trans,k):
    with Solver(name="z3") as s:
        
        trace = [declare(i) for i in range(k)]
        
        #semantics
        s.add_assertion(init(trace[0]))
        for i in range(k-1):
            s.add_assertion(trans(trace[i], trace[i+1]))
            
        if s.solve():
            '''
            m = s.get_model()
            for n, v in m:
                print(f'{n} = {v}')
            '''
            for i in range(k):
                print("Passo ",i)
                for v in trace[i]:
                    print(v, "=", s.get_value(trace[i][v]))
                print("------------------------")
    
                        
####
gera_traco(declare,init,trans,5)

Passo  0
m = -1
V = 20.0
v = 20.0
t = 0.0
timer = 0.0
------------------------
Passo  1
m = 0
V = 20.0
v = 20.0
t = 0.0
timer = 0.0
------------------------
Passo  2
m = 0
V = 20.0
v = 3602879701896397/36028797018963968
t = 3602879701896397/36028797018963968
timer = 3602879701896397/36028797018963968
------------------------
Passo  3
m = 1
V = 20.0
v = 3602879701896397/36028797018963968
t = 3602879701896397/36028797018963968
timer = 0.0
------------------------
Passo  4
m = 3
V = 0.0
v = 0.0
t = 3602879701896397/36028797018963968
timer = 0.0
------------------------


### Modelação em lógica temporal linear das propriedades

1. O veículo imobiliza-se completamente em menos de $t$ segundos

In [6]:
def imobiliza(state):
    return Implies(GE(state['t'], Real(1)), And(Equals(state['V'] , Real(0)), Equals(state['m'], Stopped)))


def bmc_always(declare,init,trans,inv,K):
    for k in range(1,K+1):
        s = Solver()
        traco = [declare(i) for i in range(k)]
        s.add_assertion(init(traco[0]))
        for i in range(k-1):
            s.add_assertion(trans(traco[i], traco[i+1]))
            
        s.add_assertion(Not(inv(traco[k-1])))
        
        if s.solve():
            for i in range(k):
                print("Passo ",i)
                for v in trace[i]:
                    print(v, "=", s.get_value(trace[i][v]))
                print("------------------------")
            print("A propriedade nao e invariante")
            return
    return "A propriedade é válida para traços de tamanho até " + str(K)

bmc_always(declare,init,trans,imobiliza,5)

NonLinearError: ((((V0 - v0) * 10.0) + -500.0) * (t1 - t0))

2. A velocidade $V$ diminui sempre com o tempo

In [ ]:
def diminui(s, p):
    return Implies(p['t'] > s['t'], p['V'] < s['V'])

def bmc_always(declare,init,trans,inv,K):
    for k in range(1,K+1):
        s = Solver()
        traco = [declare(i) for i in range(k)]
        s.add_assertion(init(traco[0]))
        for i in range(k-1):
            s.add_assertion(trans(traco[i], traco[i+1]))
            
        s.add_assertion(Not(inv(traco[k-2],traco[k-1])))
        
        if s.solve():
            for i in range(k):
                print("Passo ",i)
                for v in trace[i]:
                    print(v, "=", s.get_value(trace[i][v]))
                print("------------------------")
            print("A propriedade nao e invariante")
            return
    return "A propriedade é válida para traços de tamanho até " + str(K)

bmc_always(declare,init,trans,diminui,5)